In [ ]:
import functions_framework
import pandas as pd
import requests
from pytz import timezone 
from datetime import datetime
from keys import gcp
from keys import api
import sqlalchemy


def collect_weather_data(request):
    schema = "gans_global" # The name of your database
    host = "35.187.50.113"
    user = "root"
    password = gcp # Your gcp password
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    cities_df = pd.read_sql("cities", con=connection_string)


    # As we will be working in the cloud, computers can be located anywhere
    # Let's adjust the computer timezone to our local timezone

    berlin_timezone = timezone('Europe/Berlin')
    API_key = api
 #USE YOUR OWN OPEN WHEATHER KEY HERE

    weather_items = []

    for _, city in cities_df.iterrows():
        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["City_id"]

        url = (f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric")
        response = requests.get(url)
        weather_data = response.json()

        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

        for item in weather_data["list"]:
            weather_item= {
            "city_id": city_id,
            "forecast_time": item.get("dt_txt", None),
            "outlook": item["weather"][0].get("description", None),
            "temperature": item["main"].get("temp", None),
            "rain_in_last_3h": item.get("rain", {}).get("3h", 0),
            "wind_speed": item["wind"].get("speed", None),
            "rain_prob": item.get("pop", None),
            "data_retrieved_at": retrieval_time
        }

            weather_items.append(weather_item)

    weather_df = pd.DataFrame(weather_items)
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df["data_retrieved_at"] = pd.to_datetime(weather_df["data_retrieved_at"])

    weather_df.to_sql('weather', if_exists='append',con=connection_string,index=False)

    return 'Data has been updated'
  
  


  
